# How to simulate the LUCJ ansatz using matrix product states

Following from the previous guide, we show how to use ffsim to simulate the [LUCJ ansatz](../explanations/lucj.ipynb) using matrix product states. In this way, we can calculate an approximation to the LUCJ energy, which is itself an approximation to the ground state energy, for an ethene molecule. This is particularly useful in complicated cases, such as for large molecules, where even the LUCJ energy cannot be computed exactly. 

As before, let's start by building the molecule.

In [7]:
import pyscf
import pyscf.mcscf

import ffsim

# Build an ethene molecule
bond_distance = 1.339
a = 0.5 * bond_distance
b = a + 0.5626
c = 0.9289
mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    symmetry="d2h",
)

# Define active space
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
scf = pyscf.scf.RHF(mol).run()
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Compute FCI energy
mol_data.run_fci()

print(f"norb = {norb}")
print(f"nelec = {nelec}")

converged SCF energy = -77.8266321248745
Parsing /tmp/tmp961hod15
converged SCF energy = -77.8266321248745
CASCI E = -77.8742165643863  E(CI) = -4.02122442107773  S^2 = 0.0000000
norb = 4
nelec = (2, 2)


/home/bart/PycharmProjects/ffsim/.ffsim/lib/python3.10/site-packages/pyscf/gto/mole.py:1286: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)


Since our molecule has a closed-shell Hartree-Fock state, we'll use the spin-balanced variant of the UCJ ansatz, [UCJOpSpinBalanced](../api/ffsim.rst#ffsim.UCJOpSpinBalanced). We'll initialize the ansatz from t2 amplitudes obtained from a CCSD calculation and we'll restrict same-spin interactions to a line topology, and opposite-spin interactions to those within the same spatial orbital, which allows the ansatz to be simulated directly on a square lattice.

The following code cell initializes the LUCJ ansatz operator.

In [8]:
from pyscf import cc

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    scf,
    frozen=[i for i in range(mol.nao_nr()) if i not in active_space],
).run()

# Construct LUCJ operator
n_reps = 1
pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)

lucj_operator = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    ccsd.t2, n_reps=n_reps, interaction_pairs=interaction_pairs
)

E(CCSD) = -77.87421536374038  E_corr = -0.04758323886585098


## Convert the Hamiltonian to a matrix product operator (MPO)

Currently, our Hamiltonian is an instance of the `MolecularHamiltonian` class. Using the `to_mpo` class method, we can convert this to a TeNPy MPO, which respects the fermionic symmetries. We can now use this MPO object as in the TeNPy documentation. For example, the attribute `chi` tells us the MPO bond dimension, which is an important indicator for how complicated the Hamiltonian is an MPO representation. Optionally, we can pass the `decimal_places` argument to the `to_mpo` class method, which rounds the precision of the one-body and two-body tensors. This reduces the MPO bond dimension at the expense of simulation accuracy.

In [9]:
print("original Hamiltonian type = ", type(mol_hamiltonian))
hamiltonian_mpo = mol_hamiltonian.to_mpo()
print("converted Hamiltonian type = ", type(hamiltonian_mpo))
print("maximum MPO bond dimension = ", max(hamiltonian_mpo.chi))

original Hamiltonian type =  <class 'ffsim.hamiltonians.molecular_hamiltonian.MolecularHamiltonian'>
converted Hamiltonian type =  <class 'tenpy.networks.mpo.MPO'>
maximum MPO bond dimension =  54


## Construct the LUCJ circuit as a matrix product state (MPS)

Currently, our wavefunction ansatz operator is an instance of the `UCJOpSpinBalanced` class. In a future guide, we will show how to write this as a Qiskit circuit, which is based on qubit gates. In this guide, we will use this ansatz operator to construct our wavefunction as a TeNPy MPS, which respects the fermionic symmetries. Behind the scenes, this executes the ansatz as a fermionic circuit using the TEBD algorithm. We can pass the `trunc_params` dictionary and `norm_tol` to the `lucj_circuit_as_mps` function to control the accuracy of our MPS approximation. These parameters are detailed in the TeNPy documentation. The most important keys in `trunc_params` are `chi_max`, which sets the maximum bond dimension, and `svd_min`, which sets the minimum Schmidt value cutoff. Additionally, the `norm_tol` parameter sets the maximum norm error above which the wavefunction is recanonicalized.     

In [10]:
from ffsim.tenpy.circuits.lucj_circuit import lucj_circuit_as_mps

trunc_params = {"chi_max": 16, "svd_min": 1e-6}
psi_mps = lucj_circuit_as_mps(norb, nelec, lucj_operator, trunc_params, norm_tol=1e-5)
print("wavefunction type = ", type(psi_mps))

wavefunction type =  <class 'tenpy.networks.mps.MPS'>


## Compare the energies

Now that we have converted our `MolecularHamilonian` to an MPO, and our LUCJ ansatz to an MPS, we can contract the tensors to compute the energy. In order of increasing accuracy, we can compare the LUCJ (MPS) energy, the LUCJ energy, and the FCI energy.

In [11]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister

# Compute the LUCJ (MPS) energy
lucj_mps_energy = hamiltonian_mpo.expectation_value_finite(psi_mps)
print("LUCJ (MPS) energy = ", lucj_mps_energy)

# Compute the LUCJ energy
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(lucj_operator), qubits)
lucj_state = ffsim.qiskit.final_state_vector(circuit).vec
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
lucj_energy = np.real(np.vdot(lucj_state, hamiltonian @ lucj_state))
print("LUCJ energy = ", lucj_energy)

# Print the FCI energy
fci_energy = mol_data.fci_energy
print("FCI energy = ", fci_energy)

LUCJ (MPS) energy =  -77.84651018653355
LUCJ energy =  -77.82366375743965
FCI energy =  -77.87421656438633
